In [1]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


In [2]:

# Load the pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/home/user/Meet_Patel/new1/newVenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:

def validate_aadhar(image_path, name, dob, gender, aadhar_number):
    image = Image.open(image_path).convert("RGB")
    correct_text = f"An Indian Aadhar card with name {name}, date of birth {dob}, gender {gender}, and Aadhar number {aadhar_number}."
    wrong_texts = [
        f"An Indian Aadhar card with name {name}, date of birth {dob}, gender {gender}, and a different Aadhar number.",
        f"An Indian Aadhar card with a different name, date of birth {dob}, gender {gender}, and Aadhar number {aadhar_number}.",
        f"An Indian Aadhar card with name {name}, a different date of birth, gender {gender}, and Aadhar number {aadhar_number}.",
        f"An Indian Aadhar card with name {name}, date of birth {dob}, a different gender, and Aadhar number {aadhar_number}."
    ]
    
    texts = [correct_text] + wrong_texts
    inputs = processor(text=texts, images=[image] * len(texts), return_tensors="pt", padding=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    
    correct_score = probs[0, 0].item()
    max_wrong_score = probs[0, 1:].max().item()
    
    threshold = 0.5  # Adjust this threshold based on your requirements
    is_valid = (correct_score > threshold) and (correct_score > max_wrong_score)
    
    return is_valid, correct_score, max_wrong_score


In [19]:
# Example usage
result, correct_score, max_wrong_score = validate_aadhar("sample_image1.jpeg", "Abhishek Tiwari", "07-05-2022", "Male", "8173 2997 3675")
print("Validation result:", result)
print("Correct score:", correct_score)
print("Max wrong score:", max_wrong_score)

Validation result: False
Correct score: 0.1796620786190033
Max wrong score: 0.29989534616470337
